## Convolutional Neural Networks

Although programming frameworks make convolutions easy to use, they remain one of the hardest concepts to understand in Deep Learning. A convolution layer transforms an input volume into an output volume of different size, as shown below. 

<img src="images/conv_nn.png" style="width:350px;height:200px;">

**Notation**:
- Superscript $[l]$ denotes an object of the $l^{th}$ layer. 
    - Example: $a^{[4]}$ is the $4^{th}$ layer activation. $W^{[5]}$ and $b^{[5]}$ are the $5^{th}$ layer parameters.


- Superscript $(i)$ denotes an object from the $i^{th}$ example. 
    - Example: $x^{(i)}$ is the $i^{th}$ training example input.
    
    
- Subscript $i$ denotes the $i^{th}$ entry of a vector.
    - Example: $a^{[l]}_i$ denotes the $i^{th}$ entry of the activations in layer $l$, assuming this is a fully connected (FC) layer.
    
    
- $n_H$, $n_W$ and $n_C$ denote respectively the height, width and number of channels of a given layer. If you want to reference a specific layer $l$, you can also write $n_H^{[l]}$, $n_W^{[l]}$, $n_C^{[l]}$. 
- $n_{H_{prev}}$, $n_{W_{prev}}$ and $n_{C_{prev}}$ denote respectively the height, width and number of channels of the previous layer. If referencing a specific layer $l$, this could also be denoted $n_H^{[l-1]}$, $n_W^{[l-1]}$, $n_C^{[l-1]}$. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt 

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

## Outline of the Assignment

We will be implementing the following:

- Convolution functions, including:
    - Zero Padding
    - Convolve window 
    - Convolution forward
    - Convolution backward (optional)
- Pooling functions, including:
    - Pooling forward
    - Create mask 
    - Distribute value
    - Pooling backward (optional)
    
This notebook will implement these functions from scratch in `numpy` as we did before. Then, we will use the TensorFlow equivalents of these functions to build the following model:

<img src="images/model.png" style="width:800px;height:300px;">

**Note**: For every forward function, there is a corresponding backward equivalent. Hence, at every forward step module we will store some parameters in a cache. These parameters are used to compute gradients during backpropagation. 

### Zero-Padding

Zero-padding adds zeros around the border of an image:

<center><img src="images/PAD.png" style="width:600px;height:400px;"></center>
<caption><center> <u> <font color='purple'> <b>Figure 1</b> </u><b>Zero-Padding</b><br> Image (3 channels, RGB) with a padding of 2. </center></caption></font>


The main benefits of padding are:

- It allows to use a CONV layer without necessarily shrinking the height and width of the volumes. This is important for building deeper networks, since otherwise the height/width would shrink as you go to deeper layers. An important special case is the "same" convolution, in which the height/width is exactly preserved after one layer. 

- It keeps more of the information at the border of an image. Without padding, very few values at the next layer would be affected by pixels at the edges of an image.

### Zero_pad
Pads all the images of a batch of examples X with zeros with [np.pad](https://docs.scipy.org/doc/numpy/reference/generated/numpy.pad.html).

In [2]:
def zero_pad(X, pad):
    """
    Pad with zeros all images of the dataset X. The padding is applied to the height and width of an image, 
    as illustrated in Figure 1.
    
    Argument:
    X -- python numpy array of shape (m, n_H, n_W, n_C) representing a batch of m images
    pad -- integer, amount of padding around each image on vertical and horizontal dimensions
    
    Returns:
    X_pad -- padded image of shape (m, n_H + 2 * pad, n_W + 2 * pad, n_C)
    """
    
    X_pad = np.pad(X, ((0, 0), (pad, pad), (pad, pad), (0, 0)), 
                   mode='constant', constant_values = (0,0))
    
    return X_pad

### Single Step of Convolution 

In this part, implement a single step of convolution, in which you apply the filter to a single position of the input. This will be used to build a convolutional unit, which: 

- Takes an input volume 
- Applies a filter at every position of the input
- Outputs another volume (usually of different size)

<center><img src="images/Convolution_schematic.gif" style="width:500px;height:300px;"></center>
<caption><center> <u> <font color='purple'> <b>Figure 2</b> </u> <b>Convolution operation</b><br> with a filter of 3x3 and a stride of 1 (stride = amount you move the window each time you slide) </font></center></caption>

In a computer vision application, each value in the matrix on the left corresponds to a single pixel value. You convolve a 3x3 filter with the image by multiplying its values element-wise with the original matrix, then summing them up and adding a bias. We will implement a single step of convolution, corresponding to applying a filter to just one of the positions to get a single real-valued output. 

Later we'll apply this function to multiple positions of the input to implement the full convolutional operation. 
    
### Conv_single_step

In [4]:
def conv_single_step(a_slice_prev, W, b):
    """
    Apply one filter defined by parameters W on a single slice (a_slice_prev) of the output activation 
    of the previous layer.
    
    Arguments:
    a_slice_prev -- slice of input data of shape (f, f, n_C_prev)
    W -- Weight parameters contained in a window - matrix of shape (f, f, n_C_prev)
    b -- Bias parameters contained in a window - matrix of shape (1, 1, 1)
    
    Returns:
    Z -- a scalar value, the result of convolving the sliding window (W, b) on a slice x of the input data
    """

    s = np.multiply(a_slice_prev, W)
    Z = np.sum(s)
    Z += float(b)

    return Z

In [2]:
def initialize_matrix(nh):

    matrix = [[1, 0, -1] for _ in range(0, nh)]
    return matrix

In [16]:
A_input = [[ 30, 0, 10] for _ in range(0,  3)]
A = np.matrix(A_input)
A

matrix([[30,  0, 10],
        [30,  0, 10],
        [30,  0, 10]])

In [24]:


matrix_filter = initialize_matrix(2)
filter = np.matrix(matrix_filter)

filter

matrix([[ 1,  0, -1],
        [ 1,  0, -1]])

In [25]:
def forward_prop(A, filter):
    
    nA = A.shape[0]
    nF = filter.shape[0]

    # Matriz para armazenar o resultado
    output_dim = nA - nF + 1
    output = np.zeros((output_dim, output_dim))

    # Aplicando o filtro na matriz A
    for i in range(output_dim):
        for j in range(output_dim):
            sub_matrix = A[i:i+nF, j:j+nF]
            output[i, j] = np.sum(np.multiply(sub_matrix, filter))

    return output

forward_prop(A, filter)

ValueError: operands could not be broadcast together with shapes (2,2) (2,3) 

In [ ]:
A_ = A[:filter.shape[1]]
for row in A_:
    print (row)

[[30  0 10]]
[[30  0 10]]
[[30  0 10]]


In [ ]:
forward_prop(A, filter)

array([[60.]])

In [ ]:
def convulational_layer(A_input, nh, nw, np):
    
    np.matrix(nh, nw)

    np.w
    
    for c in A_input:
        c 


